In [1]:
import sys
import json
import time
import traceback
import requests
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup,element
import numpy as np
import cv2
from PIL import Image
from io import BytesIO
import base64
from mss import mss
from input_args import get_input_args
from remote_server.remote_selenium_server import selenium_server
from utility_functions import *

In [34]:
class TinderBotV1():
    def __init__(self,driver):
        self.url = 'https://tinder.com/'
        self.driver = driver
    
    def login(self, human_login = False):
        driver = self.driver
        driver.get(self.url)
        if human_login:
            wait_to_find(driver,'//*[@id="q243527110"]/div/div[1]/div/main/div[1]/div/div',500)
            time.sleep(15)
        else:
            find_and_click(driver,'//*[@id=q243527110]/div/div[1]/div/main/div[1]/div/div/div/div/header/div/div[2]/div[2]/a/div[2]/div[2]')
            find_and_click(driver,'//*[@id="q-1484853966"]/main/div/div[1]/div/div/div[3]/span/div[3]/button')
            find_and_type(driver,'//*[@id="q-1484853966"]/main/div/div[1]/div/div[2]/div/input','4047190217')
            find_and_click(driver,'//*[@id="q-1484853966"]/main/div/div[1]/div/button')
            print('Enter your Tinder SMS verification code:')
            sms_code = input()
            find_and_type(driver,'//*[@id="q-1484853966"]/main/div/div[1]/div/div[3]/input[1]',sms_code)
            find_and_click(driver, '//*[@id="q-1484853966"]/main/div/div[1]/div/button')
            time.sleep(1)
            find_and_click(driver, '//*[@id="q-1484853966"]/main/div/div[1]/div/div[2]/button')
            find_and_click(driver, '//*[@id="q-1484853966"]/main/div/div[1]/div/div[4]/input[1]')
            print('Enter your Tinder email verification code:')
            email_code = input()
            time.sleep(1)
            find_and_type(driver,'//*[@id="q-1484853966"]/main/div/div[1]/div/div[4]/input[1]',email_code)
            find_and_click(driver, '//*[@id="q-1484853966"]/main/div/div/div/div[3]/button[1]')
            find_and_click(driver, '//*[@id="q-1484853966"]/main/div/div/div/div[3]/button[2]')
    
    def enter_verification_code(self,xpaths):
        driver = self.driver
        code = input()
        find_and_type(driver,xpaths[0],code)
        find_and_click(driver, xpaths[1])
        time.sleep(1)
        exists = check_exists_by_xpath(driver,'//*[@id="codeVerificationErrorMessage"]/div')
        if exists:
            print('Wrong Code!')
            self.enter_verification_code(xpaths)

    def get_match_images(self,api=False):
        driver = self.driver
        if api:
            local_storage = driver.execute_script('return window.localStorage;')
            api_token = local_storage['TinderWeb/APIToken']
            url = 'https://api.gotinder.com/v2/recs/core'
            data = get_tinder_api_request(url,api_token)
            for i in data['data']['results']:
                user_id = i['user']['im = im.crop((left, top, right, bottom))_id']
                photos = []
                for y in i['user']['photos']:
                    img_url = y['processedFiles'][0]['url']
                    photos.append(img_url)
            
                rating = get_rating(photos,rating=2.7)
                print(i['user']['name'])
                print(rating)
                if rating['match']:
                    res = post_tinder_api_request(f'https://api.gotinder.com/like/{user_id}?locale=en', api_token)
                else:
                    get_tinder_api_request(f'https://api.gotinder.com/pass/{user_id}?locale=en',api_token)
        else:
            loc,size = find_elem_pos_and_size(driver,'//*[@id="q243527110"]/div/div[1]/div/main/div[1]/div')
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            div = soup.select('#q243527110 > div > div.App__body.H\(100\%\).Pos\(r\).Z\(0\) > div > main > div.H\(100\%\) > div > div > div.Mt\(a\).Px\(4px\)--s.Pos\(r\).Expand.H\(--recs-card-height\)--ml.Maw\(--recs-card-width\)--ml > div.recsCardboard__cardsContainer.H\(100\%\).Pos\(r\).Z\(1\) > div > div.Toa\(n\).Bdbw\(--recs-gamepad-height\).Bdbc\(t\).Bdbs\(s\).Bgc\(\#000\).Wc\(\$transform\).Prs\(1000px\).Bfv\(h\).Ov\(h\).W\(100\%\).StretchedBox.Bdrs\(8px\) > div.Expand.D\(f\).Pos\(r\).tappable-view.Cur\(p\) > div.CenterAlign.D\(f\).Fxd\(r\).W\(100\%\).Px\(8px\).Pos\(a\).TranslateZ\(0\)')
            buttons = div[0].find_all('button')
            left = loc['x']+700
            top = loc['y']
            right = loc['x']+1000 + size['width']
            bottom = loc['y'] + size['height']+500
    
            images = []
            for i in range(1,len(buttons)+1):
                find_and_click(driver,f'//*[@id="q243527110"]/div/div[1]/div/main/div[1]/div/div/div[1]/div[1]/div/div[3]/div[1]/div[2]/button[{i}]', False)
                png = driver.get_screenshot_as_png()
                im = Image.open(BytesIO(png))
                im = im.crop((left, top, right, bottom))
                rgb_im = im.convert('RGB')
                buffered = BytesIO()
                rgb_im.save(buffered, format="JPEG")
                img_str = base64.b64encode(buffered.getvalue())
                images.append(img_str.decode('utf-8'))

            return images
    
    def match_decision(self,images,min_rating):
        driver = self.driver
        rating = get_rating(images,rating=min_rating)
        print(rating)
        if rating['match']:
            find_by_selector_and_click(driver,'#q243527110 > div > div.App__body.H\(100\%\).Pos\(r\).Z\(0\) > div > main > div.H\(100\%\) > div > div > div.Mt\(a\).Px\(4px\)--s.Pos\(r\).Expand.H\(--recs-card-height\)--ml.Maw\(--recs-card-width\)--ml > div.recsCardboard__cardsContainer.H\(100\%\).Pos\(r\).Z\(1\) > div > div.Pos\(a\).B\(0\).Iso\(i\).W\(100\%\).Start\(0\).End\(0\) > div > div.Mx\(a\).Fxs\(0\).Sq\(70px\).Sq\(60px\)--s.Bd.Bdrs\(50\%\).Bdc\(\$c-like-green\) > button')
            return 1
        else:
            find_by_selector_and_click(driver,'#q243527110 > div > div.App__body.H\(100\%\).Pos\(r\).Z\(0\) > div > main > div.H\(100\%\) > div > div > div.Mt\(a\).Px\(4px\)--s.Pos\(r\).Expand.H\(--recs-card-height\)--ml.Maw\(--recs-card-width\)--ml > div.recsCardboard__cardsContainer.H\(100\%\).Pos\(r\).Z\(1\) > div > div.Pos\(a\).B\(0\).Iso\(i\).W\(100\%\).Start\(0\).End\(0\) > div > div.Mx\(a\).Fxs\(0\).Sq\(70px\).Sq\(60px\)--s.Bd.Bdrs\(50\%\).Bdc\(\$c-pink\) > button')
            return 0

In [3]:
platform = sys.platform
# args = get_input_args()
remote = False #args.remote

chrome_options = Options()
chrome_options.add_experimental_option("prefs", { "profile.default_content_setting_values.notifications": 1})
desired_cap = chrome_options.to_capabilities()

if platform == 'darwin':
    if remote:
        selenium_server()
        driver = webdriver.Remote(command_executor='http://localhost:4444/wd/hub', desired_capabilities=desired_cap)
    else:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
elif platform == 'linux':
    if remote:
        selenium_server()
        driver = webdriver.Remote(command_executor='http://localhost:4444/wd/hub', desired_capabilities=desired_cap)
    else:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [35]:
tinder_bot = TinderBotV1(driver)

In [ ]:
tinder_bot.login(human_login=True)

In [37]:
likes = 0
while likes < 5:
    images = tinder_bot.get_match_images()
    likes += tinder_bot.match_decision(images,2.55)

{'match': True, 'race': 'white', 'rating': 2.956688642501831}
{'match': False, 'race': 'white', 'rating': 2.096468925476074}
{'match': True, 'race': 'white', 'rating': 3.069244146347046}
{'match': False, 'race': 'white', 'rating': 2.5289158821105957}
{'match': False, 'race': 'latino hispanic', 'rating': 2.2994470596313477}
{'match': True, 'race': 'white', 'rating': 2.6211965084075928}
{'match': False, 'race': 'black', 'rating': 2.4128310680389404}
{'match': True, 'race': 'black', 'rating': 2.696208953857422}
{'match': True, 'race': 'black', 'rating': 2.783353328704834}
